[View in Colaboratory](https://colab.research.google.com/github/findingfoot/ML_practice-codes/blob/master/Different_layers_in_Neural_Network_1D_and_2D_emulated_data.ipynb)

Info:
    We will be dealing with different kinds of layers in Neural Networks
    
    1. Convolution Layer
    2. Activation Layer
    3. Max Pool layer
    4. Full connected layer
    
    Kind of datasets used:
        1. 1D data emulating row wise data
        2. 2D data emulating images

In [0]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import os


In [0]:
ops.reset_default_graph()

sess = tf.Session()


#conv layer stuff
size_of_data = 25
conv_filter_size = 5
maxpool_size = 5
stride = 1

In [0]:
#reproducibility

seed = 23
np.random.seed(seed)
tf.set_random_seed(seed)

In [0]:
#Begin with 1D data

data_1d = np.random.normal(size=size_of_data)

#now create a placeholder 
x_input_1d = tf.placeholder(dtype=tf.float32, shape=[size_of_data])


In [0]:
#create the convolution

def conv_layer_1d(input_1d, my_filter, stride):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    conv_output = tf.nn.conv2d(input_4d, filter = my_filter, strides = [1,1,stride,1], padding = 'VALID')
    
    #removing the added dimension
    
    conv_output_1d = tf.squeeze(conv_output)
    return(conv_output_1d)

my_filter = tf.Variable(tf.random_normal(shape = [1,conv_filter_size,1,1]))

my_convolution_output = conv_layer_1d(x_input_1d,my_filter, stride)

In [0]:
#activation function
def activation(input_1d):
    return(tf.nn.relu(input_1d))

#creating activation layer
my_activation_output = activation(my_convolution_output)

In [0]:
#max pooling layer

def max_pool(input_1d, width, stride):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    #specify the window size
    
    pool_output = tf.nn.max_pool(input_4d, 
                                 ksize=[1,1,width,1], 
                                 strides= [1,1,stride,1],
                                padding = "VALID")
    
    #removing extra dimension
    pool_output_1d = tf.squeeze(pool_output)
    
    return(pool_output_1d)

my_max_pool_output = max_pool(my_activation_output, width = maxpool_size, stride = stride)

In [8]:
#fully connected layer

def fully_connected(input_layer, num_outputs):
    #shape of multiplication weight matrix
    print('shape of input layer',(num_outputs))
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_outputs]]))
    
    #initialize weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    
    #initialize the bias
    
    bias = tf.random_normal(shape=[num_outputs])
    
    #converting 1d array into 2d array
    input_layer_2d = tf.expand_dims(input_layer, 0)
    
    #do the matrix multiplication and add bias
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    
    #squeeze out the extra dimension
    
    full_output_1d = tf.squeeze(full_output)
    
    return (full_output_1d)

my_full_output = fully_connected(my_max_pool_output, 5)
    

shape of input layer 5


In [9]:
#lets run the graph and see

init = tf.global_variables_initializer()
sess.run(init)

feed_dict = {x_input_1d: data_1d}

print('++++++++++++++++ Running 1D data ++++++++++++++++++++++ \n')

print('Input = array of length %d ' %(x_input_1d.shape.as_list()[0]))
print('convolution with filter, length = %d, stride size = %d, results in an array of length %d : \n' 
      % (conv_filter_size, stride, my_convolution_output.shape.as_list()[0]))

print(sess.run(my_convolution_output, feed_dict=feed_dict))


print('\n\n+++++++++++ Activation Block +++++++++++++++++')

print('\n Input = above array of length %d' % (my_convolution_output.shape.as_list()[0]))
print('\n Relu activation function returns an array of length %d' %(my_activation_output.shape.as_list()[0]))
print(sess.run(my_activation_output, feed_dict=feed_dict))


print('++++++++++++++++++++++ Max Pool layer +++++++++++++++++++')
print('\n Input = above array of length %d' % (my_activation_output.shape.as_list()[0]))
print('Max Pool, window length = %d, stride size = %d, results in an array of length %d : \n' 
      % (maxpool_size,stride, my_max_pool_output.shape.as_list()[0]))
print(sess.run(my_max_pool_output, feed_dict=feed_dict))


print('\n++++++++++++++++++++++ Fully Connected Layer ++++++++++++++++++')
print('\n Input = above array of length %d' % (my_max_pool_output.shape.as_list()[0]))
print('Fully connected layer on all 4 rows with %d outputs:' % 
      (my_full_output.shape.as_list()[0]))
print(sess.run(my_full_output, feed_dict=feed_dict))

++++++++++++++++ Running 1D data ++++++++++++++++++++++ 

Input = array of length 25 
convolution with filter, length = 5, stride size = 1, results in an array of length 21 : 

[-0.08767419 -2.3798776   0.45607725  1.4182255   2.0363877   2.753027
 -3.5175762  -0.99021906 -2.4224021   1.4362352  -1.3924831   1.9778298
  2.9213955   0.40812322 -4.9096665   0.9910472   2.7992413   2.7859051
 -3.77736    -1.9265695  -1.6774355 ]


+++++++++++ Activation Block +++++++++++++++++

 Input = above array of length 21

 Relu activation function returns an array of length 21
[0.         0.         0.45607725 1.4182255  2.0363877  2.753027
 0.         0.         0.         1.4362352  0.         1.9778298
 2.9213955  0.40812322 0.         0.9910472  2.7992413  2.7859051
 0.         0.         0.        ]
++++++++++++++++++++++ Max Pool layer +++++++++++++++++++

 Input = above array of length 21
Max Pool, window length = 5, stride size = 1, results in an array of length 17 : 

[2.0363877 2.753027  

In [0]:
# Lets deal with 2D data

ops.reset_default_graph()
sess = tf.Session()

#parameters for the run

row_size = 10
col_size = 10
conv_size = 2
conv_stride_size = 2
maxpool_size = 2
maxpool_stride_size = 1

seed = 14
np.random.seed(seed)
tf.set_random_seed(seed)

In [0]:
data_size = [row_size, col_size]
data_2d  = np.random.normal(size = data_size)

x_input_2d = tf.placeholder(dtype = tf.float32, shape=data_size)

In [0]:
#convolution layer for 2d

def conv_layer_2d(input_2d, my_filter, stride_size):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    convolution_output = tf.nn.conv2d(input_4d, filter = my_filter,
                                      strides = [1, stride_size, stride_size, 1]
                                     , padding = "VALID")
    
    #removing extra dimenstions
    conv_output_2d = tf.squeeze(convolution_output)
    return(conv_output_2d)

my_filter = tf.random_normal(shape = [conv_size,conv_size,1,1], dtype = tf.float32)
my_convolution_output = conv_layer_2d(x_input_2d, my_filter=my_filter, stride_size=conv_stride_size)
    
    
#Activation function

def activation(input_1d):
    return(tf.nn.relu(input_1d))

my_activation_output = activation(my_convolution_output)

#max pool layer

def max_pool(input_2d, width, height, stride):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, height, width, 1],
                                strides = [1, stride, stride, 1],
                                padding = "VALID")
    
    pool_output_2d = tf.squeeze(pool_output)
    return(pool_output_2d)

my_maxpool_output = max_pool(my_activation_output, width=maxpool_size, height = maxpool_size,
                            stride = maxpool_stride_size)


#fully connected layer

def fully_connected(input_layer, num_outputs):
    flat_input = tf.reshape(input_layer, [-1])
    
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input),[num_outputs]]))
    
    weight = tf.random_normal(weight_shape, stddev = 0.1)
    
    bias = tf.random_normal(shape = [num_outputs])
    
    input_2d = tf.expand_dims(flat_input, 0)
    
    full_output = tf.add(tf.matmul(input_2d, weight), bias)
    
    full_output_2d = tf.squeeze(full_output)

    return(full_output_2d)

my_full_output = fully_connected(my_maxpool_output, 5)

In [13]:
init = tf.global_variables_initializer()
sess.run(init)

feed_dict = {x_input_2d: data_2d}

print('>>>> 2D Data <<<<')

# Convolution Output
print('Input = %s array' % (x_input_2d.shape.as_list()))
print('%s Convolution, stride size = [%d, %d] , results in the %s array' % 
      (my_filter.get_shape().as_list()[:2],conv_stride_size,conv_stride_size,my_convolution_output.shape.as_list()))
print(sess.run(my_convolution_output, feed_dict=feed_dict))

# Activation Output
print('\nInput = the above %s array' % (my_convolution_output.shape.as_list()))
print('ReLU element wise returns the %s array' % (my_activation_output.shape.as_list()))
print(sess.run(my_activation_output, feed_dict=feed_dict))

# Max Pool Output
print('\nInput = the above %s array' % (my_activation_output.shape.as_list()))
print('MaxPool, stride size = [%d, %d], results in %s array' % 
      (maxpool_stride_size,maxpool_stride_size,my_maxpool_output.shape.as_list()))
print(sess.run(my_maxpool_output, feed_dict=feed_dict))

# Fully Connected Output
print('\nInput = the above %s array' % (my_maxpool_output.shape.as_list()))
print('Fully connected layer on all %d rows results in %s outputs:' % 
      (my_maxpool_output.shape.as_list()[0],my_full_output.shape.as_list()[0]))
print(sess.run(my_full_output, feed_dict=feed_dict))

>>>> 2D Data <<<<
Input = [10, 10] array
[2, 2] Convolution, stride size = [2, 2] , results in the [5, 5] array
[[-2.6543946  -1.826481    2.4554284   3.346291   -1.7156737 ]
 [-1.9840114  -1.4294717   1.5124216  -3.6152885  -0.990766  ]
 [-2.912898    1.1083796  -3.3490636   0.22011638 -0.25112176]
 [ 3.4821644   3.5253954  -4.0970163   1.7628901  -0.07008743]
 [ 2.5144246   1.7911873   2.1380415  -0.7509017   2.091725  ]]

Input = the above [5, 5] array
ReLU element wise returns the [5, 5] array
[[0.         0.27592117 0.131613   0.07465228 0.        ]
 [0.         0.         0.93434703 0.         0.        ]
 [0.         1.0157608  0.09662145 0.         1.3311785 ]
 [2.8225572  0.6472102  0.         0.88419056 1.6294262 ]
 [0.6589929  1.0502551  1.5168029  0.         0.7599966 ]]

Input = the above [5, 5] array
MaxPool, stride size = [1, 1], results in [4, 4] array
[[0.3871706 2.3803105 3.1918988 3.1918988]
 [0.3871706 0.3871706 2.4946017 2.4946017]
 [1.0912018 1.0912018 2.4946017 2